In [96]:
from tmu.tsetlin_machine import TMCoalescedClassifier
import numpy as np 
import pickle
import torch
from tabulate import tabulate
import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
from itertools import combinations
# from apriori_python import apriori
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import seaborn as sns
!export OMP_NUM_THREADS=10
n_clauses_per_class = 8000
treshold = int(n_clauses_per_class * 0.8)
s_param = 11.40000000000001
# s_param = 5

"s henger sammen hvor ofte du ser pattern"
"lav s sier at man ser et pattern ofte, og kan glemme dette"
"belonning er s/(s-1)"
"random penalty er 1/s"

'random penalty er 1/s'

In [111]:
cuda_tm = TMCoalescedClassifier(n_clauses_per_class, treshold, s_param,  platform='CUDA', boost_true_positive_feedback=0, feature_negation=False)

In [110]:
game_amount = 100000
# tsetlin_filename, self_can_bet_index= "/home/jaoi/master22/pet_for_sale/winning_games_db/{}_tsetlined_games.pkl".format(game_amount), 17
tsetlin_filename, self_can_bet_index = "/home/jaoi/master22/pet_for_sale/winning_games_db/4PPO_{}_tsetlined_leadbit_games.pkl".format(game_amount), 21

with open(tsetlin_filename, "rb") as fp:
        games = pickle.load(fp)

In [112]:
# preprossesing
def remove_duplicate_pairs(games):
    new_games = []
    for i,game in enumerate(games):
        int_arr = game[0].astype(int)
        complete_arr = np.append(int_arr, game[1])
        new_games.append(complete_arr)

    new_games = np.array(new_games)
    
    x = np.random.rand(new_games.shape[1])
    y = new_games.dot(x)
    unique, index = np.unique(y, return_index=True)
    unique_xy = new_games[index]
    
    unique_pairs = []
    for game in unique_xy:
        last, rest = game[-1], game[:-1]
        unique_pairs.append(np.array((rest.astype(np.float32), last), dtype=object))
    return np.array(unique_pairs)


def oversample(info_tup, desired_amount, source_games):
    bob = []
    for game in source_games:
        if game[1] == info_tup[0]:
            bob.append(game)
    indexes = np.random.randint(0, info_tup[1], desired_amount - info_tup[1])
    temp = []
    for ind in indexes:
        temp.append(bob[ind])
    return np.array(temp)


def preprocess_games(games):
    
    where_can = []
    for i, game in enumerate(games):
        if game[0][self_can_bet_index] == 1:
            where_can.append(i)
    indexes = np.array(where_can)
    print("past indexes")
    games_can_bid = games[indexes]

    unique_pairs = remove_duplicate_pairs(games_can_bid)
    print("past unique pairs")
    ysu = Counter()
    for game in unique_pairs:
        ysu[game[1]] += 1
    
    need_of_oversample = [tup for tup in ysu.most_common() if tup[1] < 2000]
    print("past unique need of oversample")
    


    unique_oversample = np.copy(unique_pairs)
    temp = []
    for overnd in need_of_oversample:
        temp.extend(oversample(overnd, 2000, unique_pairs))
    unique_oversample = np.concatenate((unique_oversample, np.array(temp)), axis=0)
    print("past unique oversample")
    
    uo = Counter()
    for game in unique_oversample:
        uo[game[1]] += 1
    print(uo)
    return unique_oversample

unique_oversample = preprocess_games(games)

past indexes
past unique pairs
past unique need of oversample
past unique oversample
Counter({0: 134296, 4: 79133, 6: 55871, 2: 43489, 3: 38945, 8: 34897, 5: 27761, 7: 19703, 10: 13970, 9: 12721, 11: 5188, 12: 3549, 1: 2000, 13: 2000, 14: 2000, 16: 2000, 15: 2000})


In [113]:
x =np.array([game[0] for game in unique_oversample[:, :1]])
y =np.array([game[0] for game in unique_oversample[:, 1:]])

In [114]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2)
# remove self bid feature, which is at index 21
x_train = np.delete(x_train, 21, 1)
x_test = np.delete(x_test, 21, 1)

In [115]:
epochs = 1
# for epoch in range(epochs):
#     tm.fit(x_train, y_train)

# print("Accuracy tm:", 100*(tm.predict(x_test) == y_test).mean())

for epoch in range(epochs):
    cuda_tm.fit(x_train, y_train)

    print("Epoch {} Accuracy tm:".format(epoch), 100*(cuda_tm.predict(x_test) == y_test).mean())

Epoch 0 Accuracy tm: 76.66440748657526


In [116]:
def get_name_of_feature_from_4_rounds(feature_nr):
    # for own player with 4 bits representing the cards as max number is 11
    reverse_4_bits = [3, 2, 1, 0]
    reverse_5_bits = [4, 3, 2, 1, 0]
    reverse_leading_bits = [3, 2, 1]
    leading_bits = ["lead>10", "10>lead>5", "5>lead>0", "even lead", "5>trail>0", "10>trail>5", "trail>10"]
    if feature_nr in [0,1,2,3,4]:
        return "own coins bit {}".format(2**(reverse_5_bits[feature_nr]))
    elif feature_nr in [5,6,7,8]:
        return "own nr 1 card bit {}".format(2**reverse_4_bits[feature_nr - 5])
    elif feature_nr in [9,10,11,12]:
        return "own nr 2 card bit {}".format(2**reverse_4_bits[feature_nr - 9])
    elif feature_nr in [13,14,15,16]:
        return "own nr 3 card bit {}".format(2**reverse_4_bits[feature_nr - 13])
    elif feature_nr in [17,18,19,20]:
        return "own nr 4 card bit {}".format(2**reverse_4_bits[feature_nr - 17])
    elif feature_nr in [21,22,23,24,25]:
        return "current own bid {}".format(2**reverse_5_bits[feature_nr - 21])
    elif feature_nr in [26,27,28,29,30]:
        return "resulting own coins if fold {}".format(2**reverse_5_bits[feature_nr - 26])
    elif feature_nr in [31,32,33,34]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 31])
    elif feature_nr in [35,36,37]:
        return "self next turn order {}".format(reverse_leading_bits[feature_nr - 35])
    # for next player in line
    elif feature_nr in [38,39,40,41,42]:
        return "next player coins bit {}".format(2**(reverse_5_bits[feature_nr - 38]))
    elif feature_nr in [43,44,45,46]:
        return "next player nr 1 card bit {}".format(2**reverse_4_bits[feature_nr - 43])
    elif feature_nr in [47,48,49,50]:
        return "next player nr 2 card bit {}".format(2**reverse_4_bits[feature_nr - 47])
    elif feature_nr in [51,52,53,54]:
        return "next player nr 3 card bit {}".format(2**reverse_4_bits[feature_nr - 51])
    elif feature_nr in [55,56,57,58]:
        return "next player nr 4 card bit {}".format(2**reverse_4_bits[feature_nr - 55])
    elif feature_nr == 59:
        return "next can bid bit"
    elif feature_nr in [60, 61, 62, 63, 64]:
        return "current next player bid {} bit".format(2**reverse_5_bits[feature_nr - 60])
    elif feature_nr in [65, 66, 67, 68, 69]:
        return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 65])
    elif feature_nr in [70, 71, 72, 73]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 70])
    elif feature_nr in [74, 75, 76]:
        return "next player turn order {}".format(reverse_leading_bits[feature_nr - 74])
    # for last player in line
    elif feature_nr in [77,78,79,80,81]:
        return "final player coins bit {}".format(2**(reverse_5_bits[feature_nr - 77]))
    elif feature_nr in [82,83,84,85]:
        return "final player nr 1 card bit {}".format(2**reverse_4_bits[feature_nr - 82])
    elif feature_nr in [86,87,88,89]:
        return "final player nr 2 card bit {}".format(2**reverse_4_bits[feature_nr - 86])
    elif feature_nr in [90,91,92,93]:
        return "final player nr 3 card bit {}".format(2**reverse_4_bits[feature_nr - 90])
    elif feature_nr in [94,95,96,97]:
        return "final player nr 4 card bit {}".format(2**reverse_4_bits[feature_nr - 94])
    elif feature_nr == 98:
        return "final player can bid"
    elif feature_nr in [99, 100, 101, 102, 103]:
        return "current final player bid {}".format(2**reverse_5_bits[feature_nr - 99])
    elif feature_nr in [104, 105, 106, 107, 108]:
        return "resulting final player coins if fold {}".format(2**reverse_5_bits[feature_nr - 104])
    elif feature_nr in [109, 110, 111, 112]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 109])
    elif feature_nr in [113, 114, 115]:
        return "final player turn order {}".format(reverse_leading_bits[feature_nr - 113])
    # remainder stats
    elif feature_nr in [116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130]:
        return "{} stack bit on".format(feature_nr - 116)
    elif feature_nr in [131, 132, 133, 134]:
        return "board cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 131])
    elif feature_nr in [135, 136, 137, 138]:
        return "board cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 135])
    elif feature_nr in [139, 140, 141, 142]:
        return "board cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 139])
    elif feature_nr == 143:
        return "board_std"
    elif feature_nr in [144, 145, 146, 147, 148, 149, 150]:
        return leading_bits[feature_nr - 144]
    return "not implemented"

In [117]:
def show_coalesced_weight(tm, the_class, weight):
    mw = tm.weight_banks[the_class].get_weights()
    return mw[weight]


def show_coalesced_clause_4r(tm, clause):
    number_of_features = 151
    # print("Clause #%d: " % (clause), end=' ')
    l = []
    for k in range(number_of_features*2):
        if tm.get_ta_action(clause, k) == 1:
            if k < number_of_features:
                l.append(" "+get_name_of_feature_from_4_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_4_rounds(k-number_of_features))
    return " ∧ ".join(l)

In [118]:
def get_string_coalesced_clause_4r(tm, clause):
    number_of_features = 151
    # print("Clause #%d: " % (clause), end=' ')
    l = []
    for k in range(number_of_features*2):
        if tm.get_ta_action(clause, k) == 1:
            if k < number_of_features:
                l.append(get_name_of_feature_from_4_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_4_rounds(k-number_of_features))
    return l

In [119]:
def prepare_tm_for_apriori(x, y, tm, sigma=0):
    c_out = tm.transform(x.reshape(1, -1))
    clauses_out = np.where(c_out == 1)[1].astype(int)
    mw = tm.weight_banks[y].get_weights()
    weight_to_plot = mw[clauses_out]
    clauses_out_r = clauses_out[abs(weight_to_plot) > sigma]
    return clauses_out_r

In [120]:
def single_tm_apriori(x, y, tm, sigma=-100000):
    pre_ap = prepare_tm_for_apriori(x, y, tm, sigma)
    check_clause = [show_coalesced_clause_4r(tm, clause).strip().split("∧") for i, clause in enumerate(pre_ap) if show_coalesced_weight(cuda_tm, y_test[20], clause) > 0]
    new_check = []
    for clause in check_clause:
        temp = []
        for feat in clause:
            temp.append(feat.strip())
        new_check.append(temp)
    te = TransactionEncoder()
    te_ary = te.fit(new_check).transform(new_check)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    return df

In [128]:
df = single_tm_apriori(x_test[18], y_test[18], cuda_tm, sigma=0)
frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
pd.set_option('max_colwidth', 1000)
frequent_itemsets

,support,itemsets,length
0,0.131579,(board cards 1. bit 8),1
1,0.108553,(board cards 2. bit 8),1
2,0.243421,(final player turn order 2),1
3,0.125000,(next can bid bit),1
4,0.213816,(next player turn order 3),1
5,0.105263,(resulting card on fold 4),1
6,0.226974,(resulting card on fold 8),1
7,0.256579,(self next turn order 1),1
8,0.105263,"(self next turn order 1, final player turn order 2)",2


In [127]:
y_test[18]

0

In [ ]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.1) ]

In [ ]:
def get_combinations(tm, x):
    c_out = tm.transform(x.reshape(1, -1))
    clauses_out = np.where(c_out == 1)[1].astype(int)
    feature_list_per_clause = [get_string_coalesced_clause_4r(cuda_tm, clause) for i, clause in enumerate(clauses_out)]
    comb_list = []
    for feat_list in feature_list_per_clause:
        for comb in combinations(feat_list, 3):
            comb_list.append(comb)
    c = Counter(comb_list)
    print(c.most_common(10))
get_combinations(cuda_tm, x_test[20])

In [88]:
c_out = cuda_tm.transform(x_test[0].reshape(1, -1))
clauses_out = np.where(c_out == 1)[1].astype(int)
for clause in clauses_out:
    print(show_coalesced_clause_4r(cuda_tm, clause))

 next player nr 2 card bit 4 ∧  final player nr 3 card bit 1
 self next turn order 2 ∧  next player coins bit 4 ∧  final player nr 1 card bit 8 ∧  final player turn order 1 ∧  board cards 1. bit 8 ∧  board cards 2. bit 1
 next player coins bit 4 ∧  final player coins bit 2 ∧  final player turn order 2
 resulting card on fold 8
 resulting card on fold 8
 current next player bid 8 bit
 self next turn order 2 ∧  next can bid bit ∧  resulting card on fold 8 ∧  final player coins bit 1 ∧  resulting final player coins if fold 1 ∧  1 stack bit on ∧  board cards 1. bit 8 ∧  board cards 1. bit 4
 next player nr 2 card bit 1 ∧  current final player bid 4
 resulting card on fold 8 ∧  resulting card on fold 4 ∧  board cards 2. bit 4
 self next turn order 2 ∧  next can bid bit ∧  final player coins bit 1 ∧  final player turn order 1 ∧  board cards 1. bit 8
 resulting card on fold 4 ∧  next can bid bit ∧  final player nr 1 card bit 2 ∧  final player turn order 1 ∧  7 stack bit on
 self next turn ord